### <span style="color: blue;">Code reference </span>
#### <span style="color: gray;"> Reference -- Below link was used to come up working Tweeter feed code
##### https://towardsdatascience.com/how-to-build-a-real-time-twitter-analysis-using-big-data-tools-dd2240946e64

# <span style="color: green;"> Required imports to get the feed data from tweeter</span>

In [26]:
#Twitter feed related imports

# For sending GET requests from the API
import requests

# For saving access tokens and for file management when creating and adding to the dataset
import os

# For dealing with json responses we receive from the API
import json

# For displaying the data after
import pandas as pd

# For saving the response data in CSV format
import csv

# For parsing the dates received from twitter in readable formats
from datetime import datetime, timedelta
import dateutil.parser
import unicodedata

# <span style="color: green;"> To get live feed data from tweeter we need to do the following steps:</span>
#### <span style="color: gray;"> 1. Register with twitter for a developer account. https://developer.twitter.com/</span> 
#### <span style="color: gray;"> 2. Create an app within the twitter developer portal</span> 
#### <span style="color: gray;"> 3. Bearer token will be created with app creation</span> 
#### <span style="color: gray;"> 4. This Bearer token will be used to establish session with twitter to receive live feeds</span> 

In [27]:
def auth():
    os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAG7DfwEAAAAAbJLqeNiJMbEMtiCNAc6X3xqUPXE%3DcGSmQEHDbvKKSfbcVNbYrbphIGvDkeZi4wNltyU7mmbGRFvVZq'
    return os.environ['TOKEN']


In [28]:
def create_headers(token):
    headers = {"Authorization":"Bearer {}".format(bearer_token)}
    return headers

# <span style="color: green;">There are only certain apis open to developers the list of open apis available for interaction with twitter can be found in </span>
##### https://developer.twitter.com/en/docs/authentication/guides/v2-authentication-mapping

In [29]:
def create_url(key,start_date,end_date,max_results=10):
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    query_params={'query':key,
                  'start_time':start_date,
                  'end_time':end_date,
                  'max_results':max_results,
                  'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                  'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                  'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                  'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                  'next_token': {}}
    return (search_url, query_params)

In [30]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [41]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "fire lang:en"
start_time = "2022-08-28T19:46:00.000Z"
end_time = "2022-08-28T20:01:00.000Z"
#"2022-08-14T04:26:00.000Z"
max_results = 15

In [42]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])
json_str=json.dumps(json_response, indent=4, sort_keys=True)
print(json_str)

Endpoint Response Code: 200
{
    "data": [
        {
            "author_id": "967115460",
            "conversation_id": "1563979982958993408",
            "created_at": "2022-08-28T20:00:59.000Z",
            "id": "1563979982958993408",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 6410
            },
            "referenced_tweets": [
                {
                    "id": "1563545576670887937",
                    "type": "retweeted"
                }
            ],
            "reply_settings": "everyone",
            "source": "Twitter for iPhone",
            "text": "RT @Bama_Sass: My grandfather\u2019s #TeslaPlaid caught fire. The computer said the left motor malfunctioned, but said the car was drivable &amp; no\u2026"
        },
        {
            "author_id": "1405601190180790273",
            "conversation_id": "15636416

In [43]:
data = json.loads(json_str)
df = pd.DataFrame.from_dict(data,orient='index')
df.to_csv('tweet_feed.csv')